# 📱 SMS Smishing Annotation Tool

Công cụ gán nhãn tương tác để phân loại tin nhắn SMS smishing và kiểm tra metadata.

## ✨ Tính năng chính:

### 🏷️ **Chế độ 1: Label Smishing**
- Gán nhãn tin nhắn là **smishing** (lừa đảo) hoặc **not smishing** (hợp pháp)
- Chỉ hiển thị các tin nhắn chưa được gán nhãn
- Tự động lưu sau mỗi lần gán nhãn
- Theo dõi tiến độ real-time

### 🔍 **Chế độ 2: Review Metadata**
- Kiểm tra và chỉnh sửa metadata cho tất cả tin nhắn
- Cập nhật `has_url` (có chứa URL hay không)
- Cập nhật `has_phone_number` (có chứa số điện thoại hay không)
- Cập nhật `sender_type` (loại người gửi)
- Hiển thị cả tin nhắn đã và chưa gán nhãn

### 🛠️ **Công cụ hỗ trợ:**
- ↩️ **Undo**: Hoàn tác thao tác gần nhất
- ⏭️ **Skip**: Bỏ qua tin nhắn hiện tại
- ⬅️➡️ **Navigate**: Di chuyển qua lại giữa các tin nhắn
- 🔢 **Jump to**: Nhảy tới tin nhắn cụ thể theo index
- 💾 **Auto-save**: Tự động lưu vào file CSV sau mỗi thay đổi

---

## 📋 Cấu trúc dữ liệu:

| Column | Type | Description |
|--------|------|-------------|
| `content` | string | Nội dung tin nhắn SMS |
| `label` | int | Nhãn smishing: `0` = Not smishing, `1` = smishing |
| `has_url` | int | Có URL: `0` = Không, `1` = Có |
| `has_phone_number` | int | Có SĐT: `0` = Không, `1` = Có |
| `sender_type` | string | Loại người gửi: `personal_number`, `shortcode`, `brandname`, `unknown` |

---

## 🚀 Bắt đầu:

1. **Chạy Cell 1**: Import thư viện
2. **Chạy Cell 2**: Load dữ liệu
3. **Chạy Cell 3**: Khởi tạo class Annotation Tool
4. **Chạy Cell 4**: Hiển thị giao diện annotation
5. **Bắt đầu gán nhãn!** 🎉

In [40]:
# Import necessary libraries
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully!")


✓ Libraries imported successfully!


In [ ]:
# Load data
DATA_FILE = 'data/structured_sms.csv'     #thay địa chỉ tương đối vào đây

def load_data():
    """Load the SMS data from CSV file."""
    df = pd.read_csv(DATA_FILE)
    return df

def save_data(df):
    """Save the SMS data to CSV file."""
    df.to_csv(DATA_FILE, index=False)
    
# Load initial data
df = load_data()
print(f"✓ Data loaded: {len(df)} total messages")
print(f"  - Labeled: {df['label'].notna().sum()}")
print(f"  - Unlabeled: {df['label'].isna().sum()}")


✓ Data loaded: 1579 total messages
  - Labeled: 0
  - Unlabeled: 1579


In [42]:
# Annotation Tool Class - ENHANCED VERSION
class SMSAnnotationTool:
    def __init__(self, df):
        self.df = df
        self.unlabeled_indices = df[df['label'].isna()].index.tolist()
        self.current_idx = 0
        self.history = []  # Track annotation history for undo
        self.mode = 'label'  # 'label' or 'metadata'
        
        # Create UI widgets
        self.create_widgets()
        
    def create_widgets(self):
        """Create all UI widgets."""
        # Mode selector
        self.mode_selector = widgets.ToggleButtons(
            options=[('Label smishing', 'label'), ('Review Metadata', 'metadata')],
            description='Mode:',
            button_style='info',
            tooltips=['Label messages as smishing/not smishing', 'Review and edit metadata']
        )
        self.mode_selector.observe(self.on_mode_change, names='value')
        
        # Progress info
        self.progress_label = widgets.HTML(value=self.get_progress_html())
        
        # Message display area
        self.message_area = widgets.HTML(
            value=self.get_message_html(),
            layout=widgets.Layout(
                width='100%',
                border='2px solid #ddd',
                padding='15px',
                margin='10px 0'
            )
        )
        
        # === LABEL MODE BUTTONS ===
        button_layout = widgets.Layout(width='150px', height='40px')
        self.smishing_btn = widgets.Button(
            description='🚫 smishing (1)',
            button_style='danger',
            layout=button_layout,
            tooltip='Mark as smishing'
        )
        self.not_smishing_btn = widgets.Button(
            description='✅ Not smishing (0)',
            button_style='success',
            layout=button_layout,
            tooltip='Mark as not smishing'
        )
        
        # === METADATA MODE WIDGETS ===
        # has_url checkbox
        self.has_url_checkbox = widgets.Checkbox(
            value=False,
            description='Has URL',
            disabled=False,
            indent=False
        )
        
        # has_phone_number checkbox
        self.has_phone_checkbox = widgets.Checkbox(
            value=False,
            description='Has Phone',
            disabled=False,
            indent=False
        )
        
        # sender_type dropdown
        self.sender_dropdown = widgets.Dropdown(
            options=['', 'personal_number', 'shortcode', 'brandname', 'unknown'],
            value='',
            description='Sender:',
        )
        
        # Save metadata button
        self.save_metadata_btn = widgets.Button(
            description='💾 Save Metadata',
            button_style='primary',
            layout=button_layout,
            tooltip='Save metadata changes'
        )
        
        # === COMMON NAVIGATION BUTTONS ===
        self.skip_btn = widgets.Button(
            description='⏭️ Skip',
            button_style='info',
            layout=button_layout,
            tooltip='Skip this message'
        )
        self.prev_btn = widgets.Button(
            description='⬅️ Previous',
            button_style='warning',
            layout=button_layout,
            tooltip='Go to previous message'
        )
        self.next_btn = widgets.Button(
            description='➡️ Next',
            button_style='warning',
            layout=button_layout,
            tooltip='Go to next message'
        )
        self.undo_btn = widgets.Button(
            description='↩️ Undo',
            button_style='warning',
            layout=button_layout,
            tooltip='Undo last annotation'
        )
        
        # Jump to specific index
        self.jump_input = widgets.IntText(
            value=0,
            description='Jump to:',
            disabled=False,
            layout=widgets.Layout(width='150px')
        )
        self.jump_btn = widgets.Button(
            description='Go',
            button_style='info',
            layout=widgets.Layout(width='60px', height='40px')
        )
        
        # Navigation info
        self.nav_label = widgets.HTML(
            value=self.get_nav_html(),
            layout=widgets.Layout(margin='10px 0')
        )
        
        # Bind button events
        self.smishing_btn.on_click(lambda b: self.annotate(1))
        self.not_smishing_btn.on_click(lambda b: self.annotate(0))
        self.save_metadata_btn.on_click(lambda b: self.save_metadata())
        self.skip_btn.on_click(lambda b: self.skip())
        self.prev_btn.on_click(lambda b: self.previous())
        self.next_btn.on_click(lambda b: self.next_message())
        self.undo_btn.on_click(lambda b: self.undo())
        self.jump_btn.on_click(lambda b: self.jump_to())
        
        # Output area for status messages
        self.output = widgets.Output()
        
    def on_mode_change(self, change):
        """Handle mode change."""
        self.mode = change['new']
        self.update_display()
        with self.output:
            clear_output(wait=True)
            mode_name = "Label Mode" if self.mode == 'label' else "Metadata Review Mode"
            print(f"🔄 Switched to {mode_name}")
    
    def get_progress_html(self):
        """Generate HTML for progress display."""
        total = len(self.df)
        labeled = self.df['label'].notna().sum()
        unlabeled = total - labeled
        percentage = (labeled / total * 100) if total > 0 else 0
        
        # Metadata stats
        has_url_count = self.df['has_url'].notna().sum()
        has_phone_count = self.df['has_phone_number'].notna().sum()
        sender_count = self.df['sender_type'].notna().sum()
        
        return f"""
        <div style='background: #f0f0f0; padding: 15px; border-radius: 8px; margin: 10px 0;'>
            <h3 style='margin: 0 0 10px 0;'>📊 Progress Overview</h3>
            <p style='margin: 5px 0; font-size: 16px;'>
                <b>Total:</b> {total} messages | 
                <b>Labeled:</b> {labeled} ({percentage:.1f}%) | 
                <b>Remaining:</b> {unlabeled}
            </p>
            <div style='background: #ddd; height: 20px; border-radius: 10px; overflow: hidden;'>
                <div style='background: linear-gradient(to right, #4CAF50, #45a049); 
                            height: 100%; width: {percentage}%; transition: width 0.3s;'></div>
            </div>
            <p style='margin: 10px 0 5px 0; font-size: 14px; color: #666;'>
                <b>Metadata Stats:</b> URL: {has_url_count} | Phone: {has_phone_count} | Sender: {sender_count}
            </p>
        </div>
        """
    
    def get_message_html(self):
        """Generate HTML for current message display."""
        if len(self.unlabeled_indices) == 0 and self.mode == 'label':
            return "<p style='text-align: center; font-size: 18px; color: green;'>No more unlabeled messages!</p>"
        
        if self.mode == 'label':
            idx = self.unlabeled_indices[self.current_idx] if self.unlabeled_indices else 0
        else:
            # In metadata mode, show all messages
            if self.current_idx >= len(self.df):
                return "<p style='text-align: center; font-size: 18px;'>No more messages!</p>"
            idx = self.current_idx
        
        row = self.df.loc[idx]
        
        # Format metadata
        metadata = []
        has_url_val = row.get('has_url')
        has_phone_val = row.get('has_phone_number')
        sender_val = row.get('sender_type')
        
        if pd.notna(has_url_val):
            metadata.append(f"🔗 URL: {'Yes' if has_url_val == 1 else 'No'}")
        else:
            metadata.append("🔗 URL: <span style='color:orange;'>Not set</span>")
            
        if pd.notna(has_phone_val):
            metadata.append(f"📞 Phone: {'Yes' if has_phone_val == 1 else 'No'}")
        else:
            metadata.append("📞 Phone: <span style='color:orange;'>Not set</span>")
            
        if pd.notna(sender_val):
            metadata.append(f"📱 Sender: {sender_val}")
        else:
            metadata.append("📱 Sender: <span style='color:orange;'>Not set</span>")
        
        # Label info
        label_val = row.get('label')
        if pd.notna(label_val):
            label_text = "🚫 smishing" if label_val == 1 else "✅ Not smishing"
            label_color = "#f44336" if label_val == 1 else "#4CAF50"
        else:
            label_text = "⚠️ Unlabeled"
            label_color = "#ff9800"
        
        metadata_html = " | ".join(metadata)
        
        # Format content with proper escaping
        content = str(row['content']).replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        
        return f"""
        <div style='background: white; padding: 20px; border-radius: 8px;'>
            <div style='margin-bottom: 10px; padding: 8px; background: {label_color}20; border-left: 4px solid {label_color}; border-radius: 5px;'>
                <strong style='color: {label_color};'>{label_text}</strong> (Index: {idx})
            </div>
            <div style='margin-bottom: 10px; padding: 8px; background: #e3f2fd; border-radius: 5px;'>
                <small style='color: #1976d2;'>{metadata_html}</small>
            </div>
            <div style='font-size: 15px; line-height: 1.6; color: #333; background: #fafafa; padding: 10px; border-radius: 5px;'>
                {content}
            </div>
        </div>
        """
    
    def get_nav_html(self):
        """Generate HTML for navigation info."""
        if self.mode == 'label':
            if len(self.unlabeled_indices) == 0:
                return "<p style='color: green; font-size: 18px;'><b>🎉 All messages are labeled!</b></p>"
            current_pos = self.current_idx + 1
            total = len(self.unlabeled_indices)
            return f"""
            <div style='font-size: 16px; margin: 10px 0;'>
                <b>Unlabeled Message:</b> {current_pos} / {total}
            </div>
            """
        else:
            # Metadata mode
            current_pos = self.current_idx + 1
            total = len(self.df)
            return f"""
            <div style='font-size: 16px; margin: 10px 0;'>
                <b>Message:</b> {current_pos} / {total}
            </div>
            """
    
    def update_display(self):
        """Update all display elements."""
        self.progress_label.value = self.get_progress_html()
        self.nav_label.value = self.get_nav_html()
        self.message_area.value = self.get_message_html()
        
        # Update metadata widgets with current values
        if self.mode == 'metadata' and self.current_idx < len(self.df):
            row = self.df.loc[self.current_idx]
            self.has_url_checkbox.value = bool(row.get('has_url') == 1)
            self.has_phone_checkbox.value = bool(row.get('has_phone_number') == 1)
            self.sender_dropdown.value = str(row.get('sender_type')) if pd.notna(row.get('sender_type')) else ''
        
        # Enable/disable buttons based on state and mode
        if self.mode == 'label':
            has_messages = len(self.unlabeled_indices) > 0
            self.prev_btn.disabled = (self.current_idx == 0) or not has_messages
            self.next_btn.disabled = True  # Not used in label mode
        else:
            # Metadata mode
            self.prev_btn.disabled = self.current_idx == 0
            self.next_btn.disabled = self.current_idx >= len(self.df) - 1
        
        self.undo_btn.disabled = len(self.history) == 0
    
    def annotate(self, label):
        """Annotate current message with given label."""
        if len(self.unlabeled_indices) == 0:
            return
        
        idx = self.unlabeled_indices[self.current_idx]
        
        # Save to history for undo
        self.history.append({
            'action': 'label',
            'idx': idx,
            'old_label': self.df.loc[idx, 'label'],
            'new_label': label
        })
        
        # Update label
        self.df.loc[idx, 'label'] = label
        
        # Save to file
        save_data(self.df)
        
        # Remove from unlabeled list
        self.unlabeled_indices.pop(self.current_idx)
        
        # Adjust current index
        if self.current_idx >= len(self.unlabeled_indices) and self.current_idx > 0:
            self.current_idx -= 1
        
        # Show status message
        with self.output:
            clear_output(wait=True)
            label_text = "smishing" if label == 1 else "Not smishing"
            print(f"✓ Saved as {label_text}! Remaining: {len(self.unlabeled_indices)}")
        
        # Update display
        self.update_display()
    
    def save_metadata(self):
        """Save metadata changes for current message."""
        if self.current_idx >= len(self.df):
            return
        
        idx = self.current_idx
        row = self.df.loc[idx]
        
        # Get new values
        new_has_url = 1 if self.has_url_checkbox.value else 0
        new_has_phone = 1 if self.has_phone_checkbox.value else 0
        new_sender = self.sender_dropdown.value if self.sender_dropdown.value else None
        
        # Save to history
        self.history.append({
            'action': 'metadata',
            'idx': idx,
            'old_values': {
                'has_url': row.get('has_url'),
                'has_phone_number': row.get('has_phone_number'),
                'sender_type': row.get('sender_type')
            },
            'new_values': {
                'has_url': new_has_url,
                'has_phone_number': new_has_phone,
                'sender_type': new_sender
            }
        })
        
        # Update dataframe
        self.df.loc[idx, 'has_url'] = new_has_url
        self.df.loc[idx, 'has_phone_number'] = new_has_phone
        self.df.loc[idx, 'sender_type'] = new_sender
        
        # Save to file
        save_data(self.df)
        
        # Move to next message
        if self.current_idx < len(self.df) - 1:
            self.current_idx += 1
        
        with self.output:
            clear_output(wait=True)
            print(f"✓ Metadata saved! (Index: {idx})")
        
        self.update_display()
    
    def skip(self):
        """Skip current message."""
        if self.mode == 'label':
            if len(self.unlabeled_indices) == 0:
                return
            self.current_idx = (self.current_idx + 1) % len(self.unlabeled_indices)
        else:
            if self.current_idx < len(self.df) - 1:
                self.current_idx += 1
        
        with self.output:
            clear_output(wait=True)
            print(f"⏭️ Skipped. Moved to next message.")
        
        self.update_display()
    
    def previous(self):
        """Go to previous message."""
        if self.mode == 'label':
            if len(self.unlabeled_indices) == 0 or self.current_idx == 0:
                return
            self.current_idx -= 1
        else:
            if self.current_idx > 0:
                self.current_idx -= 1
        
        with self.output:
            clear_output(wait=True)
            print(f"⬅️ Moved to previous message.")
        
        self.update_display()
    
    def next_message(self):
        """Go to next message (metadata mode)."""
        if self.mode == 'metadata' and self.current_idx < len(self.df) - 1:
            self.current_idx += 1
            with self.output:
                clear_output(wait=True)
                print(f"➡️ Moved to next message.")
            self.update_display()
    
    def jump_to(self):
        """Jump to specific index."""
        target = self.jump_input.value
        if target < 0 or target >= len(self.df):
            with self.output:
                clear_output(wait=True)
                print(f"⚠️ Invalid index. Must be between 0 and {len(self.df)-1}")
            return
        
        if self.mode == 'label':
            # Find position in unlabeled_indices
            if target in self.unlabeled_indices:
                self.current_idx = self.unlabeled_indices.index(target)
            else:
                with self.output:
                    clear_output(wait=True)
                    print(f"⚠️ Index {target} is already labeled or doesn't exist.")
                return
        else:
            self.current_idx = target
        
        with self.output:
            clear_output(wait=True)
            print(f"✓ Jumped to index {target}")
        
        self.update_display()

    def undo(self):
        """Undo the last annotation."""
        if len(self.history) == 0:
            with self.output:
                clear_output(wait=True)
                print("⚠️ No annotation to undo.")
            return
        
        # Get last action from history
        last_action = self.history.pop()
        idx = last_action['idx']
        
        if last_action['action'] == 'label':
            # Restore old label
            self.df.loc[idx, 'label'] = last_action['old_label']
            
            # Add back to unlabeled list if it was unlabeled before
            if pd.isna(last_action['old_label']):
                self.unlabeled_indices.insert(self.current_idx, idx)
        
        elif last_action['action'] == 'metadata':
            # Restore old metadata values
            old_vals = last_action['old_values']
            self.df.loc[idx, 'has_url'] = old_vals['has_url']
            self.df.loc[idx, 'has_phone_number'] = old_vals['has_phone_number']
            self.df.loc[idx, 'sender_type'] = old_vals['sender_type']
        
        # Save to file
        save_data(self.df)
        
        with self.output:
            clear_output(wait=True)
            print(f"↩️ Undone! Restored previous state.")
        
        self.update_display()
    
    def display(self):
        """Display the annotation tool UI."""
        # Label mode buttons
        label_button_box = widgets.HBox(
            [self.not_smishing_btn, self.smishing_btn],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )
        
        # Metadata mode widgets
        metadata_box = widgets.VBox([
            widgets.HTML(value="<h4 style='margin: 10px 0;'>Edit Metadata:</h4>"),
            self.has_url_checkbox,
            self.has_phone_checkbox,
            self.sender_dropdown,
            self.save_metadata_btn
        ], layout=widgets.Layout(padding='10px', border='1px solid #ddd', border_radius='5px'))
        
        # Navigation buttons
        nav_button_box = widgets.HBox(
            [self.prev_btn, self.skip_btn, self.next_btn, self.undo_btn],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )
        
        # Jump controls
        jump_box = widgets.HBox(
            [self.jump_input, self.jump_btn],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )
        
        # Conditional display based on mode
        mode_content = widgets.Output()
        
        def update_mode_display():
            with mode_content:
                clear_output(wait=True)
                if self.mode == 'label':
                    display(label_button_box)
                else:
                    display(metadata_box)
        
        self.mode_selector.observe(lambda change: update_mode_display(), names='value')
        
        # Initial display
        with mode_content:
            if self.mode == 'label':
                display(label_button_box)
            else:
                display(metadata_box)
        
        ui = widgets.VBox([
            self.mode_selector,
            self.progress_label,
            self.nav_label,
            self.message_area,
            mode_content,
            nav_button_box,
            jump_box,
            self.output
        ])
        
        display(ui)

print("✓ Enhanced Annotation tool class created!")

✓ Enhanced Annotation tool class created!


## 🎯 Hướng dẫn sử dụng

Chạy cell dưới đây để khởi động công cụ annotation. Bạn sẽ thấy giao diện có 2 chế độ:

---

### 📝 **CHẾ ĐỘ 1: LABEL SMISHING**

Sử dụng chế độ này để gán nhãn tin nhắn là smishing hoặc không:

#### Các nút chức năng:

| Nút | Chức năng | Mô tả |
|-----|-----------|-------|
| ✅ **Not smishing (0)** | Gán nhãn hợp pháp | Tin nhắn này KHÔNG phải smishing |
| 🚫 **smishing (1)** | Gán nhãn lừa đảo | Tin nhắn này LÀ smishing |
| ⏭️ **Skip** | Bỏ qua | Bỏ qua tin nhắn này, chuyển sang tin kế tiếp |
| ⬅️ **Previous** | Quay lại | Xem lại tin nhắn chưa gán nhãn trước đó |
| ↩️ **Undo** | Hoàn tác | Hoàn tác thao tác gán nhãn vừa rồi |
| 🔢 **Jump to** | Nhảy tới | Nhảy tới tin nhắn cụ thể theo index |

#### Quy trình gán nhãn:
1. Đọc nội dung tin nhắn
2. Xem thông tin metadata (URL, Phone, Sender)
3. Quyết định: smishing hay not smishing?
4. Click nút tương ứng → **Tự động lưu & chuyển tin kế tiếp**

---

### 🔍 **CHẾ ĐỘ 2: REVIEW METADATA**

Sử dụng chế độ này để kiểm tra/chỉnh sửa metadata của tin nhắn:

#### Các trường cần review:

| Trường | Loại | Giá trị | Mô tả |
|--------|------|---------|-------|
| **Has URL** | Checkbox | ☑️/☐ | Tin nhắn có chứa link/URL không? |
| **Has Phone** | Checkbox | ☑️/☐ | Tin nhắn có chứa số điện thoại không? |
| **Sender** | Dropdown | personal_number<br>shortcode<br>brandname<br>unknown | Loại người gửi tin nhắn |

#### Giải thích Sender Type:
- **personal_number**: Số điện thoại cá nhân (10-11 số, VD: 0912345678)
- **shortcode**: Mã ngắn dịch vụ (4-6 số, VD: 8077, 9029)
- **brandname**: Tên thương hiệu (chữ cái, VD: MoMo, Viettel)
- **unknown**: Không xác định được

#### Quy trình review metadata:
1. Chuyển sang chế độ **"Review Metadata"**
2. Đọc nội dung tin nhắn
3. Kiểm tra từng trường:
   - Có URL trong tin nhắn không? → Tick/Untick **Has URL**
   - Có số điện thoại không? → Tick/Untick **Has Phone**
   - Người gửi thuộc loại nào? → Chọn **Sender type**
4. Click **💾 Save Metadata** → Tự động lưu & chuyển tin kế tiếp

---

### 💡 **MẸO SỬ DỤNG:**

✅ **Làm việc hiệu quả:**
- Gán nhãn `label` trước (Mode 1), sau đó review metadata (Mode 2)
- Sử dụng **Skip** khi không chắc chắn, quay lại sau
- Dùng **Jump to** để kiểm tra tin nhắn cụ thể

✅ **Tránh mất dữ liệu:**
- Mọi thay đổi được **tự động lưu ngay lập tức**
- Có thể đóng notebook và mở lại bất cứ lúc nào
- Dữ liệu lưu tại: `data/structured_sms.csv`

✅ **Xử lý sai sót:**
- Nhấn **Undo** để hoàn tác ngay lập tức
- Hoặc chuyển sang **Review Metadata** mode để sửa lại

---

### 📊 **Theo dõi tiến độ:**

Phần **Progress Overview** ở đầu giao diện hiển thị:
- Tổng số tin nhắn
- Số lượng đã gán nhãn (%)
- Số lượng còn lại
- Thống kê metadata đã điền

---

**Chạy cell bên dưới để bắt đầu!**

In [43]:
# Initialize and display the annotation tool
tool = SMSAnnotationTool(df)
tool.display()


## 🛠️ Công cụ bổ sung

Sử dụng các cell dưới đây cho các tác vụ hỗ trợ:

### 📊 Cell 7: Kiểm tra thống kê hiện tại
Xem tổng quan tiến độ gán nhãn và phân bố dữ liệu

### 🔄 Cell 8: Reload dữ liệu
Tải lại dữ liệu từ file (hữu ích khi có thay đổi từ bên ngoài)

### 💾 Cell 9: Export dữ liệu đã gán nhãn
Xuất ra file riêng chỉ chứa các tin nhắn đã được gán nhãn

In [44]:
# Check current statistics
df_current = load_data()
total = len(df_current)
labeled = df_current['label'].notna().sum()
unlabeled = total - labeled
smishing_count = (df_current['label'] == 1).sum()
not_smishing_count = (df_current['label'] == 0).sum()

print("=" * 50)
print("CURRENT STATISTICS")
print("=" * 50)
print(f"Total messages:     {total}")
print(f"Labeled:            {labeled} ({labeled/total*100:.1f}%)")
print(f"  - smishing (1):       {smishing_count}")
print(f"  - Not smishing (0):   {not_smishing_count}")
print(f"Unlabeled:          {unlabeled} ({unlabeled/total*100:.1f}%)")
print("=" * 50)


CURRENT STATISTICS
Total messages:     1579
Labeled:            0 (0.0%)
  - smishing (1):       0
  - Not smishing (0):   0
Unlabeled:          1579 (100.0%)


In [45]:
# Reload data and restart tool (if needed)
# Run this cell if you want to restart the annotation tool with fresh data

df = load_data()
print(f"Data reloaded: {len(df)} messages")
print(f"Unlabeled: {df['label'].isna().sum()}")
print("\nTo start annotating, create a new tool instance:")
print("tool = SMSAnnotationTool(df)")
print("tool.display()")


Data reloaded: 1579 messages
Unlabeled: 1579

To start annotating, create a new tool instance:
tool = SMSAnnotationTool(df)
tool.display()


In [46]:
# Export labeled data only (optional)
# Uncomment and run to export only the labeled messages

# df_labeled = df_current[df_current['label'].notna()]
# df_labeled.to_csv('data/labeled_sms_only.csv', index=False)
# print(f"Exported {len(df_labeled)} labeled messages to data/labeled_sms_only.csv")
